In [3]:
# 실행하면 .env 파일의 OPENAI_API_KEY 를 자동으로 참조함.
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

chat = ChatOpenAI()

b = chat.predict("How many planets are there?")

b

'There are currently eight known planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.'

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1) #temperature 0~1까지 1에 가까울 수록 무작위성 증가

template = PromptTemplate.from_template(
    "What is the distance between {country_a} and {country_b}",
)

prompt = template.format(country_a="Mexico", country_b="Thailand")

chat.predict(prompt)

'The distance between Mexico and Thailand is approximately 16,000 kilometers (9,942 miles).'

In [5]:
template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert. And you only reply in {language}."),
        ("ai", "Ciao, mi chiamo {name}!"),
        (
            "human",
            "What is the distance between {country_a} and {country_b}. Also, what is your name?",
        ),
    ]
)

prompt = template.format_messages(
    language="Greek",
    name="Socrates",
    country_a="Mexico",
    country_b="Thailand",
)


chat.predict_messages(prompt)

AIMessage(content='Γεια σου! Το όνομά μου είναι Σωκράτης. Η απόσταση μεταξύ του Μεξικού και της Ταϊλάνδης είναι περίπου 17.000 χιλιόμετρα.')

#3.3 OutputParser and LCEL

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1)

In [2]:
from langchain.schema import BaseOutputParser


class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))

In [3]:
template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase.Do NOT reply with anything else.",
        ),
        ("human", "{question}"),
    ]
)

In [4]:
chain = template | chat | CommaOutputParser() # chain은 template, chat, CommaOutputParser를 하나로 묶어서 순서대로 실행하는 파이프라인

chain.invoke({"max_items": 5, "question": "What are the pokemons?"})

['pikachu', 'charizard', 'bulbasaur', 'squirtle', 'jigglypuff']

#3.4 Chaining Chains

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1, # temperature 0~1까지 1에 가까울 수록 무작위성 증가
    streaming=True, # streaming=True로 설정하면 응답을 스트리밍 방식으로 받을 수 있음
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

chef_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients.",
        ),
        ("human", "I want to cook {cuisine} food."),
    ]
)

chef_chain = chef_prompt | chat

In [ ]:
veg_chef_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it.",
        ),
        ("human", "{recipe}"),
    ]
)


veg_chain = veg_chef_prompt | chat # veg_chain은 veg_chef_prompt와 chat을 하나로 묶어서 순서대로 실행하는 파이프라인

final_chain = {"recipe": chef_chain} | veg_chain

final_chain.invoke({"cuisine": "indian"})
# 입력: {"cuisine": "indian"}
# 1단계: chef_chain이 인도 요리 레시피를 생성
# 2단계: 생성된 레시피가 veg_chain으로 전달되어 채식 버전으로 변환
# 출력: 채식 인도 요리 레시피

Great choice! Indian cuisine is known for its rich flavors and aromatic spices. Here's a recipe for a classic Indian dish called Butter Chicken:

Ingredients:
- 500g boneless chicken, cut into bite-sized pieces
- 2 tablespoons butter
- 1 onion, finely chopped
- 2 cloves of garlic, minced
- 1-inch piece of ginger, grated
- 2 teaspoons garam masala
- 1 teaspoon turmeric powder
- 1 teaspoon chili powder (adjust according to your spice preference)
- 1 cup tomato puree
- 1/2 cup heavy cream
- Salt, to taste
- Fresh cilantro, for garnish

Instructions:
1. Heat the butter in a large pan over medium heat. Add the chopped onion and sauté until it turns golden brown.
2. Add the minced garlic and grated ginger to the pan. Cook for another minute until fragrant.
3. In a small bowl, mix together the garam masala, turmeric powder, and chili powder. Add this spice mixture to the pan and cook for a minute to release the flavors.
4. Add the chicken pieces to the pan and cook until they are lightly brow

AIMessageChunk(content="Great choice! Butter Chicken is a delicious and popular Indian dish. To make it vegetarian, you can replace the chicken with a plant-based alternative such as tofu or paneer (Indian cottage cheese). Here's how you can modify the recipe:\n\nIngredients:\n- 500g tofu or paneer, cut into bite-sized pieces\n- 2 tablespoons butter (or vegan butter for a vegan version)\n- 1 onion, finely chopped\n- 2 cloves of garlic, minced\n- 1-inch piece of ginger, grated\n- 2 teaspoons garam masala\n- 1 teaspoon turmeric powder\n- 1 teaspoon chili powder (adjust according to your spice preference)\n- 1 cup tomato puree\n- 1/2 cup heavy cream (or coconut cream for a vegan version)\n- Salt, to taste\n- Fresh cilantro, for garnish\n\nInstructions:\n1. Heat the butter in a large pan over medium heat. Add the chopped onion and sauté until it turns golden brown.\n2. Add the minced garlic and grated ginger to the pan. Cook for another minute until fragrant.\n3. In a small bowl, mix toget

#4.1 FewShotPromptTemplate

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate

chat = ChatOpenAI(
    temperature=0.1, # 낮은 temperature로 일관성 있는 답변 생성
    streaming=True, # 스트리밍으로 실시간 답변 출력
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}") # 예시 프롬프트 템플릿 생성

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt, # 각 예시의 형식
    examples=examples, # 학습용 예시 데이터
    suffix="Human: What do you know about {country}?", # 실제 질문 템플릿
    input_variables=["country"], # 입력 변수 정의
)
# Few-Shot Learning의 장점 : AI가 특정 도메인에서 일관된 형식으로 답변해야 할 때 매우 유용
# 일관된 형식: 예시를 통해 원하는 답변 형식을 명확히 지정
# 적응성: 새로운 입력에 대해서도 학습된 패턴 적용
# 제어 가능성: 예시를 통해 답변 스타일과 내용 구조 제어

chain = prompt | chat

chain.invoke({"country": "Turkey"})

AI:
        I know this:
        Capital: Ankara
        Language: Turkish
        Food: Kebab and Baklava
        Currency: Turkish Lira

AIMessageChunk(content='AI:\n        I know this:\n        Capital: Ankara\n        Language: Turkish\n        Food: Kebab and Baklava\n        Currency: Turkish Lira')

#4.2 FewShotChatMessagePromptTemplate

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatMessagePromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


examples = [
    {
        "country": "France", # 질문에 들어갈 변수
        "answer": """ 
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What do you know about {country}?"), # 사용자 메시지
        ("ai", "{answer}"), # AI의 답변 메시지
    ]
)

# 앞서 정의한 example_prompt 형식과 examples 데이터를 조합하여 Few-shot 템플릿을 생성
example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert, you give short answers."),
        example_prompt, # Few-shot 예시들이 여기에 삽입됨
        ("human", "What do you know about {country}?"),
    ]
)

chain = final_prompt | chat

chain.invoke({"country": "Thailand"})


        I know this:
        Capital: Bangkok
        Language: Thai
        Food: Pad Thai and Tom Yum
        Currency: Thai Baht
        

AIMessageChunk(content='\n        I know this:\n        Capital: Bangkok\n        Language: Thai\n        Food: Pad Thai and Tom Yum\n        Currency: Thai Baht\n        ')

#4.3 LengthBasedExampleSelector

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import example_selector
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector.base import BaseExampleSelector


chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


class RandomExampleSelector(BaseExampleSelector):
    # 예시 데이터를 초기화
    def __init__(self, examples):
        self.examples = examples

    # 새로운 예시를 동적으로 추가
    def add_example(self, example):
        self.examples.append(example)

    # 핵심 메서드 - 입력에 따라 사용할 예시를 선택
    def select_examples(self, input_variables):
        from random import choice

        return [choice(self.examples)]


example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

example_selector = RandomExampleSelector(
    examples=examples,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector, # 선택적 예시 사용
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

prompt.format(country="Brazil")

'Human: What do you know about Italy?\nAI:\n        I know this:\n        Capital: Rome\n        Language: Italian\n        Food: Pizza and Pasta\n        Currency: Euro\n        \n\nHuman: What do you know about Brazil?'

#4.4 Serialization and Composition

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


####################

from langchain.prompts import load_prompt

prompt = load_prompt("./prompt.json")
# prompt = load_prompt("./prompt.yaml")

prompt.format(country="Korea")

####################

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}
                                     
    {example}
                              
    {start}
"""
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
]


# 1. pipeline_prompts의 각 템플릿이 먼저 실행됨
# 2. 그 결과가 final_prompt의 플레이스홀더에 삽입됨
full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts,
)


chain = full_prompt | chat

chain.invoke(
    {
        "character": "Pirate",
        "example_question": "What is your location?",
        "example_answer": "Arrrrg! That is a secret!! Arg arg!!",
        "question": "What is your fav food?",
    }
)

Arrrrg matey! Me favorite grub be a hearty plate o' salted beef and hardtack! Aye, nothin' beats a good ol' pirate feast on the high seas! Arrrrg!

AIMessageChunk(content="Arrrrg matey! Me favorite grub be a hearty plate o' salted beef and hardtack! Aye, nothin' beats a good ol' pirate feast on the high seas! Arrrrg!")

#4.5 Caching

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

set_llm_cache(SQLiteCache("cache.db")) # OpenAI API 호출 결과를 SQLite 데이터베이스에 저장하여 중복 요청을 방지


chat = ChatOpenAI(
    temperature=0.1,
    # streaming=True,
    # callbacks=[
    #     StreamingStdOutCallbackHandler(),
    # ],
)

chat.predict("How do you make italian pasta")

'To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- Pinch of salt\n\nHere is a step-by-step guide to making Italian pasta:\n\n1. On a clean work surface, pour the flour and create a well in the center.\n2. Crack the eggs into the well and add a pinch of salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it is smooth and elastic.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, roll out the dough using a pasta machine or a rolling pin until it is thin.\n7. Cut the dough into your desired shape, such as fettuccine or spaghetti.\n8. Cook the pasta in a large pot of salted boiling water for 2-3 minutes or until al dente.\n9. Drain the pasta and toss it with your favorite sauce or toppings.\n\nEnjoy your homemade Italian pasta!'

In [4]:
chat.predict("How do you make italian pasta")

'To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- Pinch of salt\n\nHere is a step-by-step guide to making Italian pasta:\n\n1. On a clean work surface, pour the flour and create a well in the center.\n2. Crack the eggs into the well and add a pinch of salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it is smooth and elastic.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, roll out the dough using a pasta machine or a rolling pin until it is thin.\n7. Cut the dough into your desired shape, such as fettuccine or spaghetti.\n8. Cook the pasta in a large pot of salted boiling water for 2-3 minutes or until al dente.\n9. Drain the pasta and toss it with your favorite sauce or toppings.\n\nEnjoy your homemade Italian pasta!'